In [ ]:
import pytz
import datetime
import itertools
import numpy as np
import pandas as pd
from ruamel.yaml import YAML
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import NumeralTickFormatter
from bokeh.models.tools import HoverTool
from bokeh.palettes import Category10, Category20
from bokeh.resources import INLINE
import panel as pn

pn.extension()
pn.config.sizing_mode = 'stretch_width'

In [ ]:
ar = pytz.timezone('America/Argentina/Buenos_Aires')
now = datetime.datetime.now(ar)
dt_string = now.strftime("%d-%m-%Y %H:%M")

In [ ]:
header = pn.pane.Markdown(f"""
# COVID-19 -- Argentina

[Volver a la página anterior](https://epassaro.github.io/covid-19) | Página siguiente: [Vacunas aplicadas en Argentina](https://epassaro.github.io/ar/vacunas)

""", width=800, margin=(0, 0, 10, 0)
)

In [ ]:
footer = pn.pane.Markdown(f"""
Datos provistos por [Sistemas Mapache](https://github.com/SistemasMapache/Covid19arData),
[Ministerio de Salud de la Nación](http://datos.salud.gob.ar/dataset/vacunas-contra-covid-19-dosis-aplicadas-en-la-republica-argentina)
e [INDEC](https://www.indec.gob.ar/indec/web/Nivel4-Tema-2-24-85).
El código fuente se distribuye en [GitHub](https://github.com/epassaro/covid-19) bajo la [GNU GPLv3 License](https://raw.githubusercontent.com/epassaro/covid-19/master/LICENSE).
<br> **Última actualización: ** _{dt_string}_ UTC−03:00. Powered by [Panel](https://panel.holoviz.org/index.html) and [GitHub Pages](https://pages.github.com/).

""", width=800, margin=(0, 0, 10, 0)
)

In [ ]:
fname = 'https://docs.google.com/spreadsheets/d/16-bnsDdmmgtSxdWbVMboIHo5FRuz76DBxsz_BbsEVWA/export?format=csv&id=16-bnsDdmmgtSxdWbVMboIHo5FRuz76DBxsz_BbsEVWA&gid=0'

In [ ]:
yaml = YAML()

with open('plots.yml') as f:
    countries = yaml.load(f)

In [ ]:
from functools import reduce

def create_table(fname, countries, key):

    df = pd.read_csv(fname, index_col='fecha', parse_dates=True, dayfirst=True)
    df = df[['osm_admin_level_4', 'nue_casosconf_diff', 'nue_fallecidos_diff']]
    df.index.name = 'date'
    df.columns = ['country', 'diff', 'xdiff']
    df = df.reset_index()
    
    df_list = []
    for country in countries[key]:
        d = df.set_index('country').loc[country].reset_index(drop=True)
        d.columns = ['date', f'diff{country}', f'xdiff{country}']
        d[country] = d[f'diff{country}'].cumsum()
        d[f'x{country}'] = d[f'xdiff{country}'].cumsum()
        d[f'roll{country}'] = d[f'diff{country}'].rolling(7).mean()
        d[f'xroll{country}'] = d[f'xdiff{country}'].rolling(7).mean()
        df_list.append(d)
        
    df_merged = reduce(lambda left, right: pd.merge(left, right, 
                                                    on='date', how='outer'), df_list).fillna(0)
    
    df = df_merged.groupby('date').sum()
    df = df.reset_index()
    df['datestr'] = df['date'].map(lambda x: x.strftime('%d-%m-%Y'))
    df = df.set_index('date')

    return df

In [ ]:
def make_plots(df, name):

    n = len(df.columns) // 4
    
    if n == 2:  # dumb fix for 2 countries plot (use 3 colors palette)
        n = 3

    if n <= 10:
        palette = itertools.cycle(Category10[n])

    else:
        palette = itertools.cycle(Category20[n])

    source = ColumnDataSource(df)
    q = figure(plot_width=680, plot_height=520, name='Nuevos casos', x_axis_type='datetime')
    r = figure(plot_width=680, plot_height=520, name='Fallecidos', x_axis_type='datetime')

    hovers = []
    xhovers = []
    for country in df[ [c for c in df.columns if c[:4] not in ['date', 'diff', 'roll'] \
                                                        and not c.startswith('x') ] ]:

        color = next(palette)
        q.vbar(x='date', top=f'diff{country}', source=source,
                 legend_label=country,
                 width=datetime.timedelta(days=1),
                 line_width=0,
                 color=color,
                 alpha=0.5,
                 name=country)

        q.line(x='date', y=f'roll{country}', source=source,
                 legend_label=country,
                 line_width=1.5,
                 color=color,
                 alpha=0.75,
                 name=country)

        r.vbar(x='date', top=f'xdiff{country}', source=source,
                 legend_label=country,
                 width=datetime.timedelta(days=1),
                 line_width=0,
                 color=color,
                 alpha=0.5,
                 name=country)
        
        r.line(x='date', y=f'xroll{country}', source=source,
                 legend_label=country,
                 line_width=1.5,
                 color=color,
                 alpha=0.75,
                 name=country)


        TOOLTIPS = [('Provincia', country),
                    ('Fecha', '@{datestr}'),
                    ('Nuevos casos', f'@{{diff{country}}}{{0,0}}'),
                    ('Media móvil 7-d', f'@{{roll{country}}}{{0,0.0}}'),
                    ('Total casos', f'@{{{country}}}{{0,0}}')] # Double curly braces for names containing whitespaces
        
        xTOOLTIPS = [('Provincia', country),
                    ('Fecha', '@{datestr}'),
                    ('Nuevos fallecidos', f'@{{xdiff{country}}}{{0,0}}'),
                    ('Media móvil 7-d', f'@{{xroll{country}}}{{0,0.0}}'),
                    ('Total fallecidos', f'@{{x{country}}}{{0,0}}')] # Double curly braces for names containing whitespaces

        hover = HoverTool(tooltips=TOOLTIPS, names=[country], description=country)
        hovers.append(hover)

        xhover = HoverTool(tooltips=xTOOLTIPS, names=[country])
        xhovers.append(xhover)

    q.add_tools(*hovers)
    q.legend.click_policy = 'hide'
    q.legend.location = 'top_left'
    q.legend.label_text_font_size = '8pt'
    q.yaxis.formatter = NumeralTickFormatter(format='0,0')
    q.xaxis.axis_label = "Fecha"
    q.yaxis.axis_label = "Nuevos casos"
    q.title.text = f"Nuevos casos confirmados de COVID-19 por día ({name})"
    
    r.add_tools(*xhovers)
    r.legend.click_policy = 'hide'
    r.legend.location = 'top_left'
    r.legend.label_text_font_size = '8pt'
    r.yaxis.formatter = NumeralTickFormatter(format='0,0')
    r.xaxis.axis_label = "Fecha"
    r.yaxis.axis_label = "Nuevos fallecidos"
    r.title.text = f"Nuevos fallecidos por COVID-19 por día ({name})"

    return pn.Tabs(q, r, name=name, margin=(10, 20, 0, 20))

In [ ]:
_ = [ make_plots(create_table(fname, countries, k), k) for k in countries ]

In [ ]:
tabs = pn.Tabs(*_)

In [ ]:
pn.Column(header, 
          tabs,
          footer,
          margin=(20, 40, 0, 40)
         ).save('index.html', 
                title='COVID-19 - Argentina', 
                resources=INLINE)